In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../Data/crypto_transactions_record_20220322_185347.csv", sep=",")

In [ ]:
date_format = "%m/%d/%Y %H:%M:%S"

def parse_crypto_purchase(row):

    fee_percent = 0.029
    df_row = pd.DataFrame({'exchange_name': ['crypto.com'],
     'depot_name': ['crypto.com'],
     'transaction_date':[pd.to_datetime(row['Timestamp (UTC)']).strftime(date_format)],
     'buy_asset': [row['Currency']],
     'buy_amount':[row['Amount']],
     'sell_asset':[row['Native Currency']],
     'sell_amount':[row['Native Amount']],
     'fee_asset':[row['Native Currency']],
     'fee_amount':[round(row['Native Amount']/(1-fee_percent)*fee_percent,2)],
     'transaction_type':['trade'],
     'note':[row['Transaction Kind']]})
    
    return df_row

def parse_viban_purchase(row):
    
    df_deposit = pd.DataFrame({'exchange_name': ['crypto.com'],
     'depot_name': ['crypto.com'],
     'transaction_date':[pd.to_datetime(row['Timestamp (UTC)']).strftime(date_format)],
     'buy_asset': [row['Currency']],
     'buy_amount':[abs(row['Amount'])],
     'sell_asset':[None],
     'sell_amount':[None],
     'fee_asset':[None],
     'fee_amount':[None],
     'transaction_type':['deposit'],
     'note':[row['Transaction Kind']]})
    
    df_purchase = pd.DataFrame({'exchange_name': ['crypto.com'],
     'depot_name': ['crypto.com'],
     'transaction_date':[pd.to_datetime(row['Timestamp (UTC)']).strftime(date_format)],
     'buy_asset': [row['To Currency']],
     'buy_amount':[row['To Amount']],
     'sell_asset':[row['Currency']],
     'sell_amount':[abs(row['Amount'])],
     'fee_asset':[None],
     'fee_amount':[None],
     'transaction_type':['trade'],
     'note':[row['Transaction Kind']]})
    
    df_row = pd.concat([df_deposit, df_purchase], axis=0)
    
    return df_row

def parse_referral_card_cashback(row):
    
    df_row = pd.DataFrame({'exchange_name': ['crypto.com'],
     'depot_name': ['crypto.com'],
     'transaction_date':[pd.to_datetime(row['Timestamp (UTC)']).strftime(date_format)],
     'buy_asset': [row['Currency']],
     'buy_amount':[row['Amount']],
     'sell_asset':[None],
     'sell_amount':[None],
     'fee_asset':[None],
     'fee_amount':[None],
     'transaction_type':['airdrop'],
     'note':[row['Transaction Kind']]})
    
    return df_row

def parse_referral_gift(row):
    
    df_row = pd.DataFrame({'exchange_name': ['crypto.com'],
     'depot_name': ['crypto.com'],
     'transaction_date':[pd.to_datetime(row['Timestamp (UTC)']).strftime(date_format)],
     'buy_asset': [row['Currency']],
     'buy_amount':[row['Amount']],
     'sell_asset':[None],
     'sell_amount':[None],
     'fee_asset':[None],
     'fee_amount':[None],
     'transaction_type':['airdrop'],
     'note':[row['Transaction Kind']]})
    
    return df_row

In [ ]:
df_out = pd.DataFrame()

for i, row in df.iterrows():
    
    transaction_kind = row['Transaction Kind']
    
    if transaction_kind == 'crypto_purchase':
        df_row = parse_crypto_purchase(row)
    elif transaction_kind == 'viban_purchase':
        df_row = parse_viban_purchase(row)
    elif transaction_kind == 'referral_card_cashback':
        df_row = parse_referral_card_cashback(row)
    elif transaction_kind == 'referral_gift':
        df_row = parse_referral_gift(row)
    else:
        print(transaction_kind)
        df_row = pd.DataFrame()
        
    df_out = pd.concat([df_out, df_row], axis=0)

df_out = df_out.reset_index(drop=True).reset_index(drop=False).rename(columns={'index':'id'})
df_out['id'] = df_out['id'] + 1
df_out['linked_transaction'] = ''

In [ ]:
df_out.to_excel('../Data/cryptocom.xlsx', index=False)